In [ ]:
import qsharp
import os, sys

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from util import plot

![](atom.excalidraw.svg)

In [ ]:
%%qsharp

operation PoloniumAtomAndObserver() : Result {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    H(atom);

    // measurement simulates the observer
    let decay = M(atom);
    Reset(atom);
    decay
}

In [ ]:
results = qsharp.run("PoloniumAtomAndObserver()", shots=1000)
plot(results, title="Polonium Atom: Observed Decay Outcomes")

![](cat2.excalidraw.svg)

In [ ]:
%%qsharp

import Std.Diagnostics.DumpMachine;
operation AtomCatAndObserver() : Result[] {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    H(atom);

    // a pair of qubits pretend to be a cat
    // they are entangled with the atom
    use (cat_a, cat_b) = (Qubit(), Qubit());
    CNOT(atom, cat_a);
    CNOT(atom, cat_b);

    // measurement simulates the observer
    MResetEachZ([atom, cat_a, cat_b])
}

In [ ]:
results = qsharp.run("AtomCatAndObserver()", shots=1000)
plot(results, title="Atom and Cat Entanglement")

![](env2.excalidraw.svg)

In [ ]:
%%qsharp

operation AtomCatObserverAndEnvironment() : Result[] {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    H(atom);

    // a pair of qubits pretend to be a cat
    // they are entangled with the atom
    use (cat_a, cat_b) = (Qubit(), Qubit());
    CNOT(atom, cat_a);
    CNOT(atom, cat_b);

    // 3 more qubits pretend to be the environment, also entangled
    use env = Qubit[3];
    for i in 0..2 {
        CNOT(atom, env[i]);
    }

    // measurement simulates the observer
    MResetEachZ([atom, cat_a, cat_b] + env)
}

In [ ]:
results = qsharp.run("AtomCatObserverAndEnvironment()", shots=1000)

# note: we have 2^6 = 64 possible outcomes, so for clarity we hide the empty outcomes
plot(results, title="Atom, Cat, Observer and Environment Entanglement", hide_empty=True)